# Multiple Views

Create our project

In [ ]:
# create project
mkdir workspace2
django-admin startproject myproject workspace2
cd workspace2

This time we create a webapp with multiple views.  We will leave using models until the next tutorial.  
First create a new webapp:

In [ ]:
python manage.py startapp multiple_views
tree -I __pycache__ .

Now register our app in the master settings.py file.  
This file is quite long, but the relevant part is at present:

In [ ]:
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

Update the installed apps:

In [ ]:
sed -i -e "/INSTALLED_APPS/ a \    'multiple_views'," myproject/settings.py
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

This has registered the webapp called `multiple_views`.  
Let's create several views for our app:

In [ ]:
cat << EOF > multiple_views/views.py
from django.http import HttpResponse
from django.shortcuts import render


# Create your views here.
def index(request):
    return HttpResponse("The index")

def view1(request):
    return HttpResponse(f"This is view1")

def view2(request, param1, param2):
    return HttpResponse(f"view2: param1 = {param1}, param2 = {param2}")

def view3(request):
    return HttpResponse(f"This will use a template")
EOF

Now create the local url mappings

In [ ]:
cat << EOF > multiple_views/urls.py
from django.urls import path

from . import views

urlpatterns = [
    # firefox 'http://localhost:7000/multiple_views'
    path('', views.index, name='index'),

    #firefox 'http://localhost:7000/multiple_views/view1'
    path('view1/', views.view1, name='view1'),

    #firefox 'http://localhost:7000/multiple_views/view2/abc/def'
    path('view2/<param1>/<param2>/', views.view2, name='view2'),

    #firefox 'http://localhost:7000/multiple_views/view3'
    path('view3/', views.view3, name='view3'),
]
EOF

Link our mappings to the master url file in the `myproject` folder

In [ ]:
cat << EOF > myproject/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('admin/', admin.site.urls),
    path('multiple_views/', include('multiple_views.urls')),
]
EOF

We should now have our webapp active.  Let's perform migrations and start the server on port 7000 of the localhost.

In [ ]:
python manage.py migrate
fuser -k 7000/tcp
python manage.py runserver 7000 &

Let's use `firefox` to see the app:

In [ ]:
firefox http://localhost:7000/multiple_views

In [ ]:
firefox http://localhost:7000/multiple_views/view1

In [ ]:
firefox http://localhost:7000/multiple_views/view2/abc/def

These views return a str; this is far too restrictive.  In practice we should use a template to return a complete page. To set up the template we first create a folder for the template:

In [ ]:
mkdir -p multiple_views/templates

Now create the template

In [ ]:
cat << EOF > multiple_views/templates/mytemplate.html
mytemplate.html<br>
First Name: {{ first_name }} <br>
Last Name: {{ last_name }} <br>
EOF

This creates the structure:

In [ ]:
tree -I __pycache__ multiple_views

We can rewrite `view3` to use the template:

In [ ]:
cat << EOF >> multiple_views/views.py
def view3(request):
    context = {
        'first_name':'Chris', 
        'last_name': 'Seddon'
    }
    return render(request, "mytemplate.html", context)
EOF

Now view the template in the browser

In [ ]:
firefox http://localhost:7000/multiple_views/view3

clean up:
* kill the server
* remove workspace

In [ ]:
cd ..
fuser -k 7000/tcp
rm -r workspace2